In [92]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv("OPEN_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("langchain_api_key")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("lanchain_project") 


In [93]:
# data insection
from langchain_community.document_loaders import WebBaseLoader

In [94]:
loader =WebBaseLoader("https://en.wikipedia.org/wiki/Generative_artificial_intelligence")
docu = loader.load()
docu

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nGenerative artificial intelligence - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [95]:
# divide into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20)
docs=text_splitter.split_documents(docu)
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='Generative artificial intelligence - Wikipedia'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='Jump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='Navigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='Contribu

In [96]:
#conert to vectors
from langchain_openai import OpenAIEmbeddings
embed= OpenAIEmbeddings()
from langchain_community.vectorstores import FAISS
vectordb=FAISS.from_documents(docs,embed)
vectordb

In [97]:
query="tools in langchain"
result=vectordb.similarity_search(query)
result

[Document(id='952daccc-548e-443e-8f7b-2f16ba1f651b', metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='ChatGPT and similar tools.[139][140][better\xa0source\xa0needed]'),
 Document(id='2d8a474c-40fe-448a-b867-3a9085da7459', metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='Large language model\nModel Context Protocol\nNeural network\nPrompt engineering'),
 Document(id='0db14a86-a6be-4009-b5be-045c9317d5e3', metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='Tools\n\n\n\n\n\nTools\nmove to sidebar\nhide\n\n\n\n\t\tActions\n\t\n\n\nReadEditView history\n\n\n\n\n\n\t\tGeneral'),
 Document(id='694cc479-1b3f-

In [98]:
#retrieval chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [99]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000241DE7494D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002419E38ED50>, root_client=<openai.OpenAI object at 0x00000241DE73EF90>, root_async_client=<openai.AsyncOpenAI object at 0x000002419E38E8D0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [100]:
promt=ChatPromptTemplate.from_template(
    """Answer the following question based on the context provided.\n\n{context}\n\nQuestion: {input}\n\nAnswer:"""
)
document_chain=create_stuff_documents_chain(llm,promt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Answer the following question based on the context provided.\n\n{context}\n\nQuestion: {input}\n\nAnswer:'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000241DE7494D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002419E38ED50>, root_client=<openai.OpenAI object at 0x00000241DE73EF90>, root_async_client=<openai.AsyncOpenAI object at 0x000002419E38E8D0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), 

In [101]:
document_chain.invoke(
    {"input":query,"context":result}
)

'Langchain is a framework used to work with large language models, enabling the development of applications that leverage these models more effectively. It provides various tools and components that facilitate tasks such as managing prompts, handling outputs, and integrating with different data sources. Some tools and capabilities in Langchain include:\n\n1. **Prompt Management**: Tools for creating, managing, and optimizing prompts to enhance interaction with language models.\n\n2. **Chains and Pipelines**: Mechanisms to connect multiple processing steps, including pre- and post-processing of model inputs and outputs.\n\n3. **Integrations**: Interfaces to connect with external data sources, APIs, and databases, allowing the language model to access and utilize diverse information during processing.\n\n4. **Memory and Context Handling**: Tools to handle conversation memory and context, allowing for more coherent and relevant interactions in applications such as chatbots.\n\n5. **Custom

In [102]:
retriever = vectordb.as_retriever()
from langchain.chains import create_retrieval_chain
retriever_chain= create_retrieval_chain(retriever,document_chain)
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024198862010>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Answer the following question based on the context provided.\n\n{context}\n\nQuestion: {input}\n\nAnswer:'), additional_kwargs={})]

In [103]:
retriever_chain.invoke({"input":query,"context":result})

{'input': 'tools in langchain',
 'context': [Document(id='952daccc-548e-443e-8f7b-2f16ba1f651b', metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='ChatGPT and similar tools.[139][140][better\xa0source\xa0needed]'),
  Document(id='2d8a474c-40fe-448a-b867-3a9085da7459', metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='Large language model\nModel Context Protocol\nNeural network\nPrompt engineering'),
  Document(id='0db14a86-a6be-4009-b5be-045c9317d5e3', metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='Tools\n\n\n\n\n\nTools\nmove to sidebar\nhide\n\n\n\n\t\tActions\n\t\n\n\nReadEditView history\n\n\n\n\